In [24]:
!ls results/llm_balanced_test_dataset

meta-llama_Meta-Llama-3.1-8B-Instruct.csv
mistralai_Ministral-8B-Instruct-2410.csv
mistralai_Mistral-7B-Instruct-v0.3.csv


In [87]:
import pandas as pd

In [15]:
!ls results/llm_balanced_test_dataset

meta-llama_Meta-Llama-3.1-8B-Instruct.csv
mistralai_Ministral-8B-Instruct-2410.csv
mistralai_Mistral-7B-Instruct-v0.3.csv


In [97]:
mistral = pd.read_csv("results/llm_balanced_test_dataset/mistralai_Mistral-7B-Instruct-v0.3.csv")
llama = pd.read_csv("results/llm_balanced_test_dataset/meta-llama_Meta-Llama-3.1-8B-Instruct.csv")

In [133]:
i = 12

In [134]:
mistral.iloc[i]["original_indication"]

'c/f occlusion of transhepatic ***** stent'

In [135]:
mistral.iloc[i]["radiologist_indication"]

'*****-year-old male with history of hilar cholangiocarcinoma status post ERCP on 12/02/2022 complicated by necrotizing pancreatitis status post multiple ***** and cystogastrostomy tube placement.'

In [140]:
mistral[mistral["llm_indication"].str.contains("\*")]

,patientdurablekey,exam_type,body_group,imaging_modality,report_deid_note_key,report_text,original_indication,radiologist_indication,llm_indication
3,DC6C83AE0C757F,CT ABDOMEN/PELVIS WITH CONTRAST,Abdomen/Pelvis,CT,DC53BB784A53E5,CT ABDOMEN/PELVIS WITH CONTRAST 11/23/2023 9:...,h/o ovarian cancer now with symptoms of bloati...,History of metastatic granulosa tumor of the o...,Metastatic granulosa cell tumor of the ovary ...
17,D5907A0F10EB0A,CT ABDOMEN/PELVIS WITH CONTRAST,Abdomen/Pelvis,CT,DFA61A74F515A9,CT ABDOMEN/PELVIS WITH CONTRAST 06/27/2021 9:...,pt is in clinical trial per protocol,Lung adenocarcinoma,"Metastatic lung adenocarcinoma, ***** fusion,..."
19,DADD20E800EF9C,CT ABDOMEN/PELVIS WITH CONTRAST,Abdomen/Pelvis,CT,DB0C4EF8470696,CT ABDOMEN/PELVIS WITH CONTRAST 11/21/2023 ...,"nausea/vomiting/emesis since Friday, hx of G t...",h/o SCC of L nasopharynx.,Acute cholecystitis in a ***** y.o. male with...
33,D92528CEBCB7B8,CT ABDOMEN/PELVIS WITH AND WITHOUT CONTRAST,Abdomen/Pelvis,CT,DF282E8B6EEBAE,CT ABDOMEN/PELVIS WITH AND WITHOUT CONTRAST 0...,colon cancer with mets to liver,Recurrent metastatic left-sided colon cancer. ...,"Metastatic colon cancer (dx Oct 2021), s/p ne..."
39,DAE1DB9C015F82,CT PELVIS WITH CONTRAST,Abdomen/Pelvis,CT,DC76CBFAA4D9F3,CT PELVIS WITH CONTRAST 07/23/2020 6:28 AM ...,Hip infection (known or suspected).,Increased bloody and purulent discharge from r...,"Paraplegic male, *****-year-old, with chronic..."
41,DC5193BC65BC47,CT ABDOMEN/PELVIS WITH CONTRAST,Abdomen/Pelvis,CT,D4997AFAE653FD,CT ABDOMEN/PELVIS WITH CONTRAST 09/12/2023 ...,intra-abdominal source of infection.,"substance abuse, UTI, CKD on HD, depression, c...","Complex abdominal infection/enteric fistulas,..."
43,DAFDD3BBADD67F,MR ABDOMEN/PELVIS WITH AND WITHOUT CONTRAST 10...,Abdomen/Pelvis,MRI,D85EDCB93F2028,MR ABDOMEN/PELVIS WITH AND WITHOUT CONTRAST 10...,"***** Sarcoma s/p *****, on surveillance",***** sarcoma dx May 2016. Currently on survei...,"***** Sarcoma, diagnosed 2016 s/p chemotherap..."
59,D085A13906371B,CT ABDOMEN/PELVIS WITH CONTRAST,Abdomen/Pelvis,CT,DAD43F912ED836,CT ABDOMEN/PELVIS WITH CONTRAST 09/25/2023 ...,"Presurgical Planning, cannot go to MRI off mon...",Intracranial mass,Pineal mass with associated obstructive hydro...
71,DCFD0749948DDC,CT ABDOMEN/PELVIS WITH CONTRAST,Abdomen/Pelvis,CT,DB54A46905E8DC,CT ABDOMEN/PELVIS WITH CONTRAST 05/20/2020 8:...,known liver mets with progressive hyponatremia...,stage IV?NSCLC (poorly differentiated carcinom...,"Stage IV squamous cell carcinoma, metastatic ..."
76,D0FCEE8A2E92F1,CT ABDOMEN/PELVIS WITH CONTRAST,Abdomen/Pelvis,CT,D9A87DBBB0CB95,CT ABDOMEN/PELVIS WITH CONTRAST 06/08/2021 1:...,"***** left lap nephrectomy, w/ persistent feve...",History of prostate cancer and pelvic radiatio...,"Male patient, age ***** y.o. with history of ..."


In [139]:
llama[llama["llm_indication"].str.contains("\*")]

,patientdurablekey,exam_type,body_group,imaging_modality,report_deid_note_key,report_text,original_indication,radiologist_indication,llm_indication
12,DFA44E40F155F0,CT ABDOMEN/PELVIS WITH CONTRAST,Abdomen/Pelvis,CT,DB70216A1210C7,CT ABDOMEN/PELVIS WITH CONTRAST 01/30/2023 2:...,c/f occlusion of transhepatic ***** stent,*****-year-old male with history of hilar chol...,"Male patient, 68 years old, with a history of ..."
22,D47D3AB1EC849E,CT ABDOMEN/PELVIS WITHOUT CONTRAST,Abdomen/Pelvis,CT,D2687C9E906191,CT ABDOMEN/PELVIS WITHOUT CONTRAST 12/16/2020...,suspect intra abdominal wound.,"***** y.o.***** a history of HTN, HFpEF, *****...",A 62-year-old male with a complex medical hist...
79,D43148B066F253,CT ABDOMEN/PELVIS WITH CONTRAST,Abdomen/Pelvis,CT,D4D78A5D7740ED,CT ABDOMEN/PELVIS WITH CONTRAST 10/25/2021 6:...,CT A&P with contrast for bladder cancer staging,"71 M with a hx of HTN, CAD w/ NSTEMI 2021 Afib...",A 71-year-old male with a history of high-grad...
